In [1]:
import os
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv

load_dotenv()
os.chdir('../')
os.getcwd()

'c:\\Users\\11\\Desktop\\LCG\\Programming\\project\\wedrive'

In [6]:
# 사용자 정보
filedir1=os.getcwd()+'/data/od_uuid/2024_1/'
filedir2=os.getcwd()+'/data/od_uuid/2024_2/'

# 데이터프레임 칼럼 지정
column_name=['uuid','time_begin','time_end','origin_lat','origin_lng','destination_lat','destination_lng','distance','heart','rank']

client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))

database = client.get_database(os.getenv('DB_Collection'))
collection1=database.get_collection('raw_od_uuid_2024_2')
collection2=database.get_collection('Timefiltered_2024')

In [7]:
# 데이터프레임에 칼럼명 지정
def Make_column(csv_file):
    raw_data=pd.read_csv(csv_file, header=None)
    data=raw_data.values.tolist()
    col_data=pd.DataFrame(data, columns=column_name)
    return col_data

In [8]:
#도착시간이 식사시간(11~13, 18~20시인 데이터 필터링)
time_filter=['11', '12', '18', '19']
def time_filtering(csv_file):
    filter=csv_file.apply(lambda x: True if x['time_end'][11:13] in time_filter else False, axis=1)
    processing_csv=csv_file[filter]
    return processing_csv[['uuid', 'time_end','destination_lat', 'destination_lng']]

In [9]:
# 데이터프레임에서 데이터를 추출하여 dictionary 형태로 변경한 후, MongoDB에 데이터 삽입
def dataframe_to_dict1(dataframe):
    if len(dataframe)!=0:
        collection1.insert_many(dataframe.to_dict('records'))

def dataframe_to_dict2(dataframe):
    if len(dataframe)!=0:
        collection2.insert_many(dataframe.to_dict('records'))

In [10]:
# 디렉터리 내 파일 모두 데이터베이스에 insert
def insert_to_database(dir_path):
    for file in os.listdir(dir_path):
        if file.endswith(".csv"):
            file_path=os.path.join(dir_path, file)
            dataframe_to_dict1(Make_column(file_path))

def insert_to_database_timefilter(dir_path):
    for file in os.listdir(dir_path):
        if file.endswith(".csv"):
            file_path=os.path.join(dir_path, file)
            dataframe_to_dict2(time_filtering(Make_column(file_path)))

In [11]:
insert_to_database(filedir1)
insert_to_database(filedir2)

In [12]:
insert_to_database_timefilter(filedir1)
insert_to_database_timefilter(filedir2)